Use Python to fetch country debt profiles. Use a horizontal bar chart, the y-axis is the list of all countries in the world (for example, Japan, Korea, Philippines, China, and more), and the x-axis is the total debt in $.  Make a ranking label on the outside and the right part of the bar, followed by the number of debt$ label

In [ ]:
red terminal says {---------------------------------------------------------------------------
AssertionError                            Traceback (most recent call last)
Cell In[30], line 5
      2 import re
      4 # Initialize Wikipedia API
----> 5 wiki_wiki = wikipediaapi.Wikipedia('en')
      7 # Fetch the page listing all countries
      8 countries_page = wiki_wiki.page("List of countries by population (United Nations)")

File ~\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\wikipediaapi\__init__.py:174, in Wikipedia.__init__(self, user_agent, language, variant, extract_format, headers, extra_api_params, **request_kwargs)
    169 if user_agent is not None:
    170     default_headers.setdefault(
    171         "User-Agent",
    172         user_agent,
    173     )
--> 174 used_language, used_variant, used_user_agent = self._check_and_correct_params(
    175     language,
    176     variant,
    177     default_headers.get("User-Agent"),
    178 )
    180 default_headers["User-Agent"] = used_user_agent + " (" + USER_AGENT + ")"
    182 self.language = used_language

File ~\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\wikipediaapi\__init__.py:733, in Wikipedia._check_and_correct_params(language, variant, user_agent)
    724 """
    725 Checks the constructor parameters and throws AssertionError if they are incorrect.
    726 Otherwise, it normalises them to easy use later on.
   (...)    730 :return: tuple of language, variant, user_agent
    731 """
    732 if not user_agent or len(user_agent) < MIN_USER_AGENT_LEN:
--> 733     raise AssertionError(
    734         "Please, be nice to Wikipedia and specify user agent - "
    735         + "https://meta.wikimedia.org/wiki/User-Agent_policy. Current user_agent: '"
    736         + str(user_agent)
    737         + "' is not sufficient. "
    738         + "Use Wikipedia(user_agent='your-user-agent', language='"
    739         + (str(user_agent) or "your-language")
    740         + "')"
    741     )
    743 if not language:
    744     raise AssertionError(
    745         "Specify language. Current language: '"
    746         + str(language)
   (...)    750         + "', language='your-language')"
    751     )

AssertionError: Please, be nice to Wikipedia and specify user agent - https://meta.wikimedia.org/wiki/User-Agent_policy. Current user_agent: 'en' is not sufficient. Use Wikipedia(user_agent='your-user-agent', language='en')}